In [15]:
# load packages
import os
import geopandas as gpd
import leafmap.foliumap as leafmap
import requests
import rioxarray as rxr
import xarray
import matplotlib.pyplot as plt
import numpy as np
from folium import Map
from folium.raster_layers import ImageOverlay
import leafmap.colormaps as cm
from geocube.api.core import make_geocube
import math
import pandas as pd
import warnings
from pandas.api.types import is_numeric_dtype

In [16]:
# define print raster function to help compare reprojections
def print_raster(raster):
    print(
        f"shape: {raster.rio.shape}\n"
        f"resolution: {raster.rio.resolution()}\n"
        f"bounds: {raster.rio.bounds()}\n"
        f"sum: {raster.sum().item()}\n"
        f"CRS: {raster.rio.crs}\n"
        f"NoDataValue: {raster.rio.nodata}\n"
        f"NoDataOriginalValues: {raster.rio.encoded_nodata}\n"
    )

In [17]:
# define function to extract zonal stats
def extract_stats (dat_fp, dat_name, vector, vect_var, stat):

    # Suppress UserWarning within this function
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", UserWarning)

        # create new vect variable so don't edit data in place
        vect = vector.copy()

        # check if vect_var column is numeric
        numeric = True
        if not is_numeric_dtype(vect[vect_var]):
            vect['key'] = pd.factorize(vect[vect_var])[0]
            vect.rename(columns={vect_var: 'orig_name', 'key': vect_var}, inplace=True)
            numeric = False

        # clip data to vector layer
        dat = rxr.open_rasterio(dat_fp, masked=True
                                ).rio.clip(vect.geometry.values, vect.crs, from_disk=True)
        dat.name = dat_name

        # create output grid
        out_grid = make_geocube(
            vector_data=vect,
            measurements=[vect_var],
            like=dat
        )

        # merge the datacube with the data
        out_grid[dat_name] = (dat.dims, dat.values,
                                dat.attrs, dat.encoding)
        
        # group data by vector variable
        grouped = out_grid.drop_vars(['spatial_ref']).groupby(vect_var)

        # calculate stats
        if stat == 'sum':
            table = grouped.sum()
        if stat == 'mean':
            table = grouped.mean()

        # reset indicies and drop band
        table = table.to_dataframe()
        table.reset_index(level='band', drop=True, inplace=True)

        # reset keys to original values
        if numeric == False:
            mapping = dict(zip(vect[vect_var], vect['orig_name']))
            table.index = table.index.map(mapping)

        # return table
        return table

In [18]:
# load data layer filepaths
data_folder = './carbon_stock_data/output_layers/'
rra_filepath = './study_area/Study_Area_RRA_MOU_revised/RRA_nonoverlap_3_revised.shp'

In [19]:
# load carbon rasters in format for zonal stats function
# file paths
carbon_fp = ['./carbon_stock_data/output_layers/carbon_sothe_spawn_t_laea.tif',
             './carbon_stock_data/output_layers/soc_0_1m_t_laea.tif',
             './carbon_flux_data/output_layers/emissions_gfw_t_yr_laea.tif',
             './carbon_flux_data/output_layers/removals_gfw_t_yr_laea.tif',
             './carbon_stock_data/output_layers/carbon_sothe_spawn_t_ha_laea.tif',            
             './carbon_stock_data/output_layers/soc_0_1m_t_ha_laea.tif',
             './carbon_flux_data/output_layers/emissions_gfw_t_ha_laea.tif',
             './carbon_flux_data/output_layers/removals_gfw_t_ha_laea.tif']

carbon_names = ['carbon_t',
                'soc_t',
                'emissions_t_yr',
                'removals_t_yr',
                'carbon_t_ha',
                'soc_t_ha',
                'emissions_t_ha',
                'removals_t_ha']

stat_names = ['sum',
              'sum',
              'sum',
              'sum',
              'mean',
              'mean',
              'mean',
              'mean']

In [20]:
# load first raster and extract CRS
crs = rxr.open_rasterio(carbon_fp[0]).rio.crs

# load study area shp
rra = gpd.read_file(rra_filepath).to_crs(crs)
rra

,FN_TRADITI,FN_TRADI_1,FN_TERRITO,ENV_NAME_C,CIG_NAME_C,RATIFIED_I,RATIFIED_D,GEOMETRY_L,GEOMETRY_A,geometry
0,15,Kaska Dena Nation (Ross River and Liard),Traditional Territory,None,KASKA,None,0000/00/00,1.261018e+06,4.107575e+10,"MULTIPOLYGON (((-176918.298 770244.601, -17646..."


In [21]:
# extract zonal stats
for i in range(len(carbon_fp)):
    if i == 0:
        stats = extract_stats(dat_fp = carbon_fp[i], dat_name = carbon_names[i], vector=rra, vect_var='FN_TRADITI', stat=stat_names[i])
    else:
        stats = pd.concat([stats,
                          extract_stats(dat_fp = carbon_fp[i], dat_name = carbon_names[i], vector=rra, vect_var='FN_TRADITI', stat=stat_names[i])],
                          axis=1)
        
stats

,carbon_t,soc_t,emissions_t_yr,removals_t_yr,carbon_t_ha,soc_t_ha,emissions_t_ha,removals_t_ha
FN_TRADITI,,,,,,,,
15.0,1.005836e+08,7.304448e+08,399981.899868,2.028691e+06,26.121627,190.037808,86.184847,17.156218


In [30]:
# add area to table
stats['area_km2'] = rra.geometry.area.values / 1e+6
stats

,carbon_t,soc_t,emissions_t_yr,removals_t_yr,carbon_t_ha,soc_t_ha,emissions_t_ha,removals_t_ha,area_km2
FN_TRADITI,,,,,,,,,
15.0,1.005836e+08,7.304448e+08,399981.899868,2.028691e+06,26.121627,190.037808,86.184847,17.156218,39291.134695


In [31]:
# export to csv
stats.to_excel(
    './outputs/y2y_carbon_rra.xlsx', index=True)

# LEAFMAP BELOW

In [79]:
cm.palettes.viridis

Box({'default': ('440154', '440256', '450457', '450559', '46075a', '46085c', '460a5d', '460b5e', '470d60', '470e61', '471063', '471164', '471365', '481467', '481668', '481769', '48186a', '481a6c', '481b6d', '481c6e', '481d6f', '481f70', '482071', '482173', '482374', '482475', '482576', '482677', '482878', '482979', '472a7a', '472c7a', '472d7b', '472e7c', '472f7d', '46307e', '46327e', '46337f', '463480', '453581', '453781', '453882', '443983', '443a83', '443b84', '433d84', '433e85', '423f85', '424086', '424186', '414287', '414487', '404588', '404688', '3f4788', '3f4889', '3e4989', '3e4a89', '3e4c8a', '3d4d8a', '3d4e8a', '3c4f8a', '3c508b', '3b518b', '3b528b', '3a538b', '3a548c', '39558c', '39568c', '38588c', '38598c', '375a8c', '375b8d', '365c8d', '365d8d', '355e8d', '355f8d', '34608d', '34618d', '33628d', '33638d', '32648e', '32658e', '31668e', '31678e', '31688e', '30698e', '306a8e', '2f6b8e', '2f6c8e', '2e6d8e', '2e6e8e', '2e6f8e', '2d708e', '2d718e', '2c718e', '2c728e', '2c738e', '2b

In [83]:
# set parameters
m = leafmap.Map(width=800, height=500, google_map='HYBRID', zoom=10)
m.add_raster(biomass, colormap='viridis', layer_name='Biomass Carbon (t)', nodata=np.nan)
m

In [7]:
# add data layers
m = leafmap.Map(width=800, height=500)



m.add_shp(rra_filepath, layer_name='Study Area')
m

'./study_area/RRA_nonoverlap_3_revised_carbon_stocks.shp'